In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
from sklearn.mixture import GaussianMixture
from nltk.corpus import stopwords  # Import the stop word list
from sklearn import datasets
import pandas as pd
import os
import sys
import re
import nltk

import numpy as np
import collections
import matplotlib.pyplot as plt
import numpy as np

# WARNING : Nltk save download in your personal folder, don t forget delete them after run model.
# Download text data sets, including stop words
nltk.download('punkt')
nltk.download("stopwords")


def body_to_words(raw_body):
    # Fonction permettant de convertir un commentaire brut en chaîne de mots
    # L’entrée est une chaîne unique (mail de film brute), et
    # la sortie est une chaîne unique (mail pré-traitée)
    # Supprimer les non-lettres
    letters_only = re.sub("[^a-zA-Z]", " ", raw_body)

    # Convertir en minuscules, diviser en mots individuels
    words = letters_only.lower().split()

    # En Python, la recherche d'un ensemble est beaucoup plus rapide que la recherche
    # une liste, convertissez les mots vides en un ensemble
    stops = set(stopwords.words("french"))

    # Joindre les mots dans une chaîne séparée par un espace,
    # et retourne le résultat.
    meaningful_words = [w for w in words if w not in stops]
    return " ".join(meaningful_words)


def word_tokenizer(text):
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(t) for t in tokens if t not in stopwords.words('french')]
    return tokens


def cluster_sentences(sentences, nb_of_clusters=2):
    tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenizer,
                                       stop_words=stopwords.words('french'),
                                       max_df=0.9,
                                       min_df=0.1,
                                       lowercase=False)
    # Construit une matrice tf-idf pour les phrases
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    kmeans = KMeans(n_clusters=nb_of_clusters,
                    init='k-means++',
                    max_iter=300,
                    n_init=1)

    kmeans.fit(tfidf_matrix)
    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    print("ORDER CENTROIDS",order_centroids)
    terms = tfidf_vectorizer.get_feature_names()
    print("TERMS", terms)
    for i in range(nb_of_clusters):
        print("Cluster %d:" % i),
        for ind in order_centroids[i, :10]:
            print("%s"% terms[ind])
    
    clusters = collections.defaultdict(list)
    for i, label in enumerate(kmeans.labels_):
        print(kmeans.labels_)
        clusters[label].append(i)
    return dict(clusters)


def k_means_model(train_data_features):
    # KMeans
    km = KMeans(n_clusters=5)
    km.fit(train_data_features)
    km.predict(train_data_features)
    labels = km.labels_

    # Plotting
    fig = plt.figure(1, figsize=(7, 7))
    ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
    ax.scatter(train_data_features[:, 3], train_data_features[:, 0], train_data_features[:, 2],
               c=labels.astype(np.float), edgecolor="k", s=50)
    ax.set_xlabel("ham width")
    ax.set_ylabel("spam length")
    ax.set_zlabel("ham length")
    plt.title("K Means", fontsize=14)


def gaussian_mixture_model(train_data_features):
    # Gaussian Mixture Model
    gmm = GaussianMixture(n_components=5)
    gmm.fit(train_data_features)
    proba_lists = gmm.predict_proba(train_data_features)

    # Plotting
    colored_arrays = np.matrix(proba_lists)
    colored_tuples = [tuple(i.tolist()[0]) for i in colored_arrays]
    fig = plt.figure(1, figsize=(7, 7))
    ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
    ax.scatter(train_data_features[:, 3], train_data_features[:, 0], train_data_features[:, 2],
               c=colored_tuples, edgecolor="k", s=50)
    ax.set_xlabel("ham width")
    ax.set_ylabel("spam length")
    ax.set_zlabel("hamp length")
    plt.title("Gaussian Mixture Model", fontsize=14)

# ---MAIN----

In [ ]:
data = './dataset/manitra_mails.csv'
train = pd.read_csv(data, encoding='utf-8')

# Obtenez le nombre de mail en fonction de la taille de la colonne dataframe
num_reviews = train["body"].size

# Initialise une liste vide pour contenir les mails
clean_train_reviews = []

# Boucle sur chaque mail; créer un index i qui va de 0 à la longueur du Nombre de mail
for i in range(0, num_reviews):
    # Appelez notre fonction pour chacun et ajoutez le résultat à la liste des mails prétraité
    clean_train_reviews.append(body_to_words(str(train["body"][i])))

print("Cleaning and parsing the training set mail body...\n")
clean_train_reviews = []
for i in range(0, num_reviews):

    # Si l’index est divisible par 1000, affiche un message.
    if ((i + 1) % 1000 == 0):
        print("body %d of %d\n" % (i + 1, num_reviews))
    clean_train_reviews.append(body_to_words(str(train["body"][i])))

print("Creating the bag of words...\n")

# Initialise l'objet "CountVectorizer", qui est celui de scikit-learn outil de sac de mots. 
vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words=None,
                             max_features=5000)

# fit_transform () remplit deux fonctions: tout d’abord, cela correspond au modèle
# et apprend le vocabulaire; deuxièmement, cela transforme nos données d'entraînement
# en vecteurs de caractéristiques. L'entrée à fit_transform doit être une liste de
# chaînes.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Les tableaux Numpy sont faciles à utiliser, convertissez donc le résultat en tableau
train_data_features = train_data_features.toarray()

# Résumer les comptes de chaque mot de vocabulaire
dist = np.sum(train_data_features, axis=0)

# Clustering
nclusters = 4
clusters = cluster_sentences(clean_train_reviews, nclusters)

In [ ]:
# for cluster in range(nclusters):
#        print("cluster ", cluster, ":")
#        for i, sentence in enumerate(clusters[cluster]):
#            print("\tsentence ", i, ": ", clean_train_reviews[sentence])

In [ ]:
gaussian_mixture_model(train_data_features)

In [ ]:
k_means_model(train_data_features)